# load packages

In [1]:
DATA_DIR = '/data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/'

In [2]:
# import packages 
import scanpy as sc
import scanpy.external as sce
import anndata as anndata
from anndata import AnnData
import time
import os
import numpy as np
import seaborn as sns
import pandas as pd
import re
import h5py
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as patches
import matplotlib.colors as colors
import matplotlib.font_manager

# other packages
import glob
from collections import OrderedDict
from scipy.sparse import csr_matrix
#import diffxpy.api as de
from scanpy.external.pp import magic
from scipy.stats import zscore
import scipy.cluster.hierarchy as hc
from statistics import median
import math
from typing import Union
import phenograph
import numpy.matlib

from kneed import KneeLocator


In [3]:
#custom libraries
from scrnaseq_preprocessing_utils import *

In [4]:
#set style
sns.set_style('white')
# define colors
color_map_clusters_nogrey = ["#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
"#5A0007", "#809693", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
"#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
"#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
"#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
"#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
"#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
"#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
"#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
"#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
"#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
"#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C"]

# annotate cells

In [55]:

adata_path = '/data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/2021_PDAC_RT_Huber_Walle_raw_data_annotated_emptydrops_noribo_nomito_clustered_nodrops_doublets_nodoubs_libsize_log1p_clustered_10000hv_genes_curated_marker_genes_210216_reanalysis.h5ad'
adata_save_path = '/data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/2021_PDAC_RT_Huber_Walle_raw_data_annotated_emptydrops_noribo_nomito_clustered_nodrops_doublets_nodoubs_libsize_log1p_clustered_10000hv_genes_curated_marker_genes_210216_reanalysis_forcytopus.h5ad'
csv_save_path = '/data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195_annotations.csv'
#annotations
adata_B_path = DATA_DIR + '2021_PDAC_RT_Huber_Walle_B_scran_raw_counts_annotated_5000hv_clustered_v2_230125_annotated.h5ad'
adata_M_path = DATA_DIR + '2021_PDAC_RT_Huber_Walle_M_scran_raw_counts_annotated_5000hv_clustered_v2_230125_annotated.h5ad'
adata_TNK_path = DATA_DIR + '2021_PDAC_RT_Huber_Walle_TNK_scran_raw_counts_annotated_5000hv_clustered_v2_230125_annotated.h5ad'


adata_paths = [adata_B_path, adata_M_path, adata_TNK_path] 

make_raw= False


In [6]:
#load adata
adata = sc.read(adata_path)
adata_annotations = []

for i in adata_paths:
    adata_subset = sc.read(i)
    adata_annotations.append(adata_subset)
    del adata_subset

adata.obs_names #check whether format is barcode_sampleid

Index(['236288388524979-T15', '196174352334259-N09', '227844233476963-T19',
       '235000971515614-T23', '239932640480493-T23', '155842596263860-T03',
       '125239746026742-T18', '204762107788076-T22', '134376349133724-T13',
       '236785384151899-T20',
       ...
       '120797925625246-T09', '166288091176166-T15', '161893219780323-T18',
       '120864485590886-T10', '130185503586669-T14', '234992247389099-N08',
       '133854511090019-N02', '169014034020203-T11', '126836650171619-N11',
       '129565717587827-T17'],
      dtype='object', name='cell_ID', length=152991)

In [7]:
print(adata.raw.X)

  (0, 16154)	1.0
  (0, 17325)	1.0
  (0, 12779)	1.0
  (0, 24301)	1.0
  (0, 7167)	1.0
  (0, 13824)	2.0
  (0, 23207)	2.0
  (0, 15562)	1.0
  (0, 8281)	1.0
  (0, 21122)	1.0
  (0, 6270)	1.0
  (0, 20863)	1.0
  (0, 9379)	6.0
  (0, 1649)	1.0
  (0, 11531)	115.0
  (0, 12814)	8.0
  (0, 12600)	1.0
  (0, 12298)	1.0
  (0, 8643)	1.0
  (0, 2844)	1.0
  (0, 1755)	1.0
  (0, 2100)	1.0
  (0, 19503)	2.0
  (0, 1725)	4.0
  (0, 15063)	1.0
  :	:
  (152990, 19905)	1.0
  (152990, 1831)	1.0
  (152990, 5565)	1.0
  (152990, 15047)	1.0
  (152990, 23580)	2.0
  (152990, 6770)	1.0
  (152990, 22437)	3.0
  (152990, 1836)	1.0
  (152990, 8202)	2.0
  (152990, 1895)	1.0
  (152990, 3578)	1.0
  (152990, 14427)	1.0
  (152990, 3596)	4.0
  (152990, 20329)	3.0
  (152990, 4226)	2.0
  (152990, 23220)	1.0
  (152990, 19550)	1.0
  (152990, 12540)	1.0
  (152990, 11395)	1.0
  (152990, 9817)	2.0
  (152990, 15741)	1.0
  (152990, 10059)	1.0
  (152990, 7273)	1.0
  (152990, 15559)	1.0
  (152990, 3905)	1.0


In [8]:
adata_annotations

[AnnData object with n_obs × n_vars = 4832 × 14068
     obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'batch', 'primary_tumor', 'Limited', 'PValue_emptydrops', 'FDR_emptydrops', 'total_molecules', 'percent_mito', 'pheno_var_k10', 'pheno_var_k20', 'pheno_var_k30', 'pheno_var_k40', 'pheno_var_k50', 'pheno_var_k60', 'pheno_var_k70', 'pheno_var_k80', 'pheno_var_k90', 'pheno_var_k100', 'major_subsets', 'pheno_B_k10', 'pheno_B_k20', 'pheno_B_k30', 'pheno_B_k40', 'pheno_B_k50', 'pheno_B_k60', 'pheno_B_k70', 'pheno_B_k80', 'pheno_B_k90', 'pheno_B_k100', 'pheno_var_B_k10', 'pheno_var_B_k20', 'pheno_var_B_k30', 'pheno_var_B_k40', 'pheno_var_B_k50', 'pheno_var_B_k60', 'pheno_var_B_k70', 'pheno_var_B_k80', 'pheno_var_B_k90', 'pheno_var_B_k100', 'pheno_annotate', 'annotation_level_0', 'annotation_level_1', 'annotation_level_2', 'anno

In [9]:
#retrieve annotations from .obs element and convert into dict

annotation_subtypes_2 = []
annotation_subtypes_3 = []

for i in adata_annotations:
    annotation_subtypes_2.append(i.obs['annotation'])
    annotation_subtypes_3.append(i.obs['pheno_annotate'])
    
annotation_subtype_dicts_1 = []


annotation_subtype_dicts_2 = []
    
for i in annotation_subtypes_2:
    annotation_subtype_2_dict = i.to_dict()
    annotation_subtype_dicts_2.append(annotation_subtype_2_dict)
    
    
annotation_subtype_dicts_3 = []
    
for i in annotation_subtypes_3:
    annotation_subtype_3_dict = i.to_dict()
    annotation_subtype_dicts_3.append(annotation_subtype_3_dict)
    


In [10]:
#concatenate dictionaries order matters! sutype 1
    
annotation_subtype_dict_2 = {}

for i in annotation_subtype_dicts_2:
    annotation_subtype_dict_2.update(i)
        
annotation_subtype_dict_3 = {}

for i in annotation_subtype_dicts_3:
    annotation_subtype_dict_3.update(i)   


In [11]:
adata.obs['annotation_cytopus'] = adata.obs_names.map(annotation_subtype_dict_2)
adata.obs['pheno_annotate'] = adata.obs_names.map(annotation_subtype_dict_3)

## add non-immune annotaitons

In [20]:
leuko_list = [15,24,0,13,1,21]
leuko = ['leukocyte']*len(leuko_list)
fibro_list = [5,26,19,9,16]
fibro = ['fibroblast and myocytes']*len(fibro_list)
endothelial_list = [14,20,4,7]
endothelial = ['endothelial cell']*len(endothelial_list)
epithelial_list = [18,25,11,3,6,8,17,10,23,2,12]
epithelial = ['epithelial cell']*len(epithelial_list)
endocrine_list = [22]
endocrine = ['neuroendocrine']*len(endocrine_list)
nan_list = []
nan = ['NaN']*len(nan_list)

In [21]:
#create dictionaries
dict0 = dict(zip(leuko_list, leuko))
dict1 = dict(zip(fibro_list, fibro))
dict2 = dict(zip(endothelial_list, endothelial))
dict3 = dict(zip(epithelial_list, epithelial))
dict4 = dict(zip(endocrine_list, endocrine))



# merging of dictionaries
dict1.update(dict0)
dict1.update(dict2)
dict1.update(dict3)
dict1.update(dict4)


TIL_classes_pheno_list_ndf_dict = dict1

In [22]:
TIL_classes_pheno_list_ndf_dict

{5: 'fibroblast and myocytes',
 26: 'fibroblast and myocytes',
 19: 'fibroblast and myocytes',
 9: 'fibroblast and myocytes',
 16: 'fibroblast and myocytes',
 15: 'leukocyte',
 24: 'leukocyte',
 0: 'leukocyte',
 13: 'leukocyte',
 1: 'leukocyte',
 21: 'leukocyte',
 14: 'endothelial cell',
 20: 'endothelial cell',
 4: 'endothelial cell',
 7: 'endothelial cell',
 18: 'epithelial cell',
 25: 'epithelial cell',
 11: 'epithelial cell',
 3: 'epithelial cell',
 6: 'epithelial cell',
 8: 'epithelial cell',
 17: 'epithelial cell',
 10: 'epithelial cell',
 23: 'epithelial cell',
 2: 'epithelial cell',
 12: 'epithelial cell',
 22: 'neuroendocrine'}

In [35]:
new_pheno = []
new_annotations = []

old_pheno = list(adata.obs['pheno_annotate'])
old_annotations = list(adata.obs['annotation_cytopus'])

for i,v in enumerate(list(adata.obs['pheno_var_k60'])):
    if TIL_classes_pheno_list_ndf_dict[v] == 'leukocyte':
        new_pheno.append(old_pheno[i])
        new_annotations.append(old_annotations[i])
    else:
        new_pheno.append('all_'+str(v))
        new_annotations.append(TIL_classes_pheno_list_ndf_dict[v])

In [37]:
set(new_pheno)

{'B_0',
 'B_1',
 'B_2',
 'B_3',
 'B_4',
 'B_5',
 'B_6',
 'M_0',
 'M_1',
 'M_10',
 'M_11',
 'M_12',
 'M_13',
 'M_14',
 'M_15',
 'M_16',
 'M_17',
 'M_18',
 'M_19',
 'M_2',
 'M_20',
 'M_21',
 'M_3',
 'M_4',
 'M_5',
 'M_6',
 'M_7',
 'M_8',
 'M_9',
 'M_pheno_var_k10_38',
 'TNK_0',
 'TNK_1',
 'TNK_10',
 'TNK_11',
 'TNK_12',
 'TNK_13',
 'TNK_14',
 'TNK_15',
 'TNK_16',
 'TNK_17',
 'TNK_18',
 'TNK_19',
 'TNK_2',
 'TNK_20',
 'TNK_21',
 'TNK_22',
 'TNK_23',
 'TNK_24',
 'TNK_25',
 'TNK_3',
 'TNK_4',
 'TNK_5',
 'TNK_6',
 'TNK_7',
 'TNK_8',
 'TNK_9',
 'all_10',
 'all_11',
 'all_12',
 'all_14',
 'all_16',
 'all_17',
 'all_18',
 'all_19',
 'all_2',
 'all_20',
 'all_22',
 'all_23',
 'all_25',
 'all_26',
 'all_3',
 'all_4',
 'all_5',
 'all_6',
 'all_7',
 'all_8',
 'all_9',
 nan}

In [38]:
set(new_annotations)

{'B-T_doublet',
 'B-memory-non-switched',
 'B-memory-switched',
 'B-naive',
 'CD4-T-naive',
 'CD4-TCM',
 'CD4-TRM-TH17',
 'CD4-TRM_terminally_exhausted',
 'CD56bright-NK',
 'CD56dim-NK',
 'CD8-T',
 'CD8-T-KLRG1neg-effector_nonexhausted',
 'CD8-T-KLRG1pos-effector_nonexhausted',
 'CD8-T-KLRG1pos-effector_preexhausted',
 'CD8-TRM_nonexhausted',
 'CD8-TRM_terminally_exhausted',
 'GC-B',
 'GC-B_proliferating',
 'Gr-MDC',
 'ILC3-NCR2pos',
 'Mac',
 'PV-Mac',
 'T',
 'T_proliferating',
 'Treg',
 'c-mono',
 'cDC1',
 'cDC2',
 'cDC3',
 'endothelial cell',
 'epithelial cell',
 'fibroblast and myocytes',
 nan,
 'nc-mono',
 'neuroendocrine',
 'p-DC'}

In [39]:
adata.obs['annotations_cytopus_final'] = new_annotations

In [40]:
adata.obs['pheno_annotate_final'] = new_pheno

In [41]:
adata.obs['annotations_cytopus_final'].value_counts()

annotations_cytopus_final
epithelial cell                         62871
endothelial cell                        24038
fibroblast and myocytes                 22081
Mac                                     11487
CD8-T-KLRG1neg-effector_nonexhausted     5362
CD8-T-KLRG1pos-effector_preexhausted     4620
Treg                                     2791
c-mono                                   2191
CD4-T-naive                              2176
CD8-TRM_nonexhausted                     2034
B-memory-switched                        1464
B-naive                                  1440
neuroendocrine                           1037
cDC2                                     1030
CD4-TCM                                   877
CD8-T                                     718
CD56dim-NK                                700
B-memory-non-switched                     603
CD8-T-KLRG1pos-effector_nonexhausted      574
CD56bright-NK                             559
CD4-TRM-TH17                              536
GC-B_pro

In [43]:
adata.obs['pheno_annotate_final'].value_counts()

pheno_annotate_final
all_2                 12559
all_3                 10340
all_4                  9537
all_5                  9524
all_6                  8771
                      ...  
M_19                    114
TNK_25                   89
M_20                     79
M_21                     64
M_pheno_var_k10_38       38
Name: count, Length: 77, dtype: int64

In [44]:
adata[adata.obs['pheno_annotate_final'].isna()]

View of AnnData object with n_obs × n_vars = 424 × 20962
    obs: 'batch', 'primary_tumor', 'LogProb_emptydrops', 'PValue_emptydrops', 'FDR_emptydrops', 'Limited', 'total_molecules', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'percent_mito', 'n_counts', 'pheno', 'true_cell_emptydrops', 'empty_droplet_sig', 'doublets', 'pheno_k10', 'pheno_k20', 'pheno_k30', 'pheno_k40', 'pheno_k50', 'pheno_k60', 'pheno_k70', 'pheno_k80', 'pheno_k90', 'pheno_k100', 'pheno_var_k10', 'pheno_var_k20', 'pheno_var_k30', 'pheno_var_k40', 'pheno_var_k50', 'pheno_var_k60', 'pheno_var_k70', 'pheno_var_k80', 'pheno_var_k90', 'pheno_var_k100', 'annotation_cytopus', 'pheno_annotate', 'annotations_cytopus_final', 'pheno_annotate_final'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_

In [45]:
print('mapped annotations:', set(adata.obs['annotation_cytopus']))

mapped annotations: {'CD8-T', 'p-DC', 'CD56bright-NK', 'T_proliferating', 'CD4-TRM-TH17', 'Gr-MDC', 'CD8-T-KLRG1neg-effector_nonexhausted', 'B-memory-non-switched', 'GC-B_proliferating', 'CD4-TRM_terminally_exhausted', 'cDC1', nan, 'Treg', 'T', 'CD8-TRM_nonexhausted', 'GC-B', 'B-memory-switched', 'Mac', 'CD8-T-KLRG1pos-effector_preexhausted', 'nc-mono', 'CD8-T-KLRG1pos-effector_nonexhausted', 'B-naive', 'c-mono', 'CD8-TRM_terminally_exhausted', 'ILC3-NCR2pos', 'CD4-TCM', 'CD56dim-NK', 'CD4-T-naive', 'B-T_doublet', 'PV-Mac', 'cDC3', 'cDC2'}


In [46]:
print('mapped pheno for annotations:', set(adata.obs['pheno_annotate']))

mapped pheno for annotations: {'M_16', 'M_6', 'M_8', 'TNK_6', 'TNK_0', 'TNK_21', 'M_13', 'TNK_10', 'B_3', 'M_7', 'TNK_12', 'TNK_7', 'TNK_18', 'M_21', 'TNK_15', 'M_17', 'M_9', 'M_5', 'TNK_19', 'M_0', 'TNK_5', 'TNK_3', 'B_1', 'M_19', 'TNK_9', 'B_0', 'M_20', 'TNK_11', 'M_4', 'TNK_17', nan, 'TNK_4', 'TNK_25', 'M_11', 'M_18', 'M_15', 'TNK_8', 'B_6', 'M_12', 'B_2', 'M_3', 'TNK_20', 'TNK_13', 'TNK_23', 'M_2', 'B_4', 'TNK_22', 'M_1', 'B_5', 'TNK_1', 'TNK_2', 'M_10', 'M_14', 'TNK_14', 'TNK_24', 'TNK_16', 'M_pheno_var_k10_38'}


In [53]:
annotation_df = adata.obs[['pheno_annotate_final','annotations_cytopus_final']]
annotation_df.columns = ['cluster','annotation']
annotation_df

,cluster,annotation
cell_ID,,
236288388524979-T15,all_4,endothelial cell
196174352334259-N09,all_3,epithelial cell
227844233476963-T19,all_4,endothelial cell
235000971515614-T23,M_17,Mac
239932640480493-T23,TNK_11,CD4-T-naive
...,...,...
234992247389099-N08,all_7,endothelial cell
133854511090019-N02,all_26,fibroblast and myocytes
169014034020203-T11,M_2,Treg


In [56]:
adata.write(adata_save_path)
print('saved to:',adata_save_path)

saved to: /data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/2021_PDAC_RT_Huber_Walle_raw_data_annotated_emptydrops_noribo_nomito_clustered_nodrops_doublets_nodoubs_libsize_log1p_clustered_10000hv_genes_curated_marker_genes_210216_reanalysis_forcytopus.h5ad


In [57]:
annotation_df.to_csv(csv_save_path)
print('saved to:',csv_save_path)

saved to: /data/peer/wallet/Jupyter/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195-y-X-Jupyter/output_data/PAAD-X-Peng_Sun-X-2019-X-10.1038s41422-019-0195_annotations.csv
